# Tâche #2 : Classification d'incidents avec un réseau  récurrent et des *embeddings* Spacy

Cette tâche est similaire à la précédente et vous réutilisez les mêmes fichiers d’entraînement, de validation et de test. Cependant, vous devez utiliser des réseaux récurrents pour classifier les textes. Plus particulièrement, vous devez entraîner un réseau de neurones LSTM pour encoder les textes et une couche linéaire pour faire la classification des textes. 

Les consignes pour cette tâche sont: 
- 	Nom du notebook : rnn.ipynb
- 	Tokenisation : Utilisation de Spacy. 
- 	Plongements de mots : Ceux de Spacy. 
- 	Normalisation : Aucune normalisation. 
- 	Structure du réseau : Un réseau LSTM avec 1 seule couche pour l’encodage de textes. Je vous laisse déterminer la taille de cette couche (à expliquer). 
- 	Analyse : Comparer les résultats obtenus avec un réseau unidirectionnel et un réseau bidirectionnel. Si vous éprouvez des difficultés à entraîner les 2 réseaux dans un même notebook, faites une copie et nommez le 2e fichier rnn-bidirectionnel.ipynb.
- 	Expliquez comment les modèles sont utilisés pour faire la classification d’un texte. 
- 	Présentez clairement vos résultats et faites-en l’analyse. 


Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [19]:
# Importation des bibliothèques nécessaires
import json
import spacy
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report


In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [21]:
# Charger le modèle de langue de spaCy
nlp = spacy.load('fr_core_news_md')

## 1. Création du jeu de données
class IncidentDataset(Dataset):
    def __init__(self, file):
        with open(file, 'r') as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = int(self.data[idx]['label'])
        # Tokenisation et plongements de mots avec spaCy
        doc = nlp(text)
        embeddings = [token.vector for token in doc]
        return torch.tensor(embeddings), label




In [22]:

# Charger les données
train_data = IncidentDataset('data/incidents_train.json')
dev_data = IncidentDataset('data/incidents_dev.json')
test_data = IncidentDataset('data/incidents_test.json')

## 2. Gestion de plongements de mots (*embeddings*)

## 3. Création de modèle(s)

In [23]:
## 3. Création de modèle(s)
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, bidirectional, pooling='max'):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.pooling = pooling

    def forward(self, x):
        x, _ = self.lstm(x)
        if self.pooling == 'max':
            x = torch.max(x, dim=1)[0]  # max pooling
        elif self.pooling == 'mean':
            x = torch.mean(x, dim=1)  # mean pooling
        elif self.pooling == 'min':
            x = torch.min(x, dim=1)[0]  # min pooling
        x = self.fc(x)
        return x

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

In [24]:

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()

In [25]:
from torch.nn.utils.rnn import pad_sequence
# Dans votre DataLoader
def collate_fn(batch):
    inputs = pad_sequence([item[0] for item in batch], batch_first=True)
    labels = torch.tensor([item[1] for item in batch])
    return inputs, labels

## 5. Entraînement de modèle(s)

In [26]:

# Initialiser le LSTM avec différentes tailles de couche cachée et directions
n_classes = len(set([label for _, label in train_data]))  # nombre d'étiquettes uniques dans les données d'entraînement
for hidden_dim in [50, 100, 200]:
    for bidirectional in [False, True]:
        model = LSTM(300, hidden_dim, n_classes, 1, bidirectional).to(device)  # 300 = dimension des plongements de mots de spaCy
        # ... Entraînez et évaluez le modèle ici ...
        # Définir l'optimiseur
        optimizer = torch.optim.Adam(model.parameters())

        # Définir le DataLoader
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)
        dev_loader = DataLoader(dev_data, batch_size=32, shuffle=False, collate_fn=collate_fn)
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False, collate_fn=collate_fn)

        # Boucle d'entraînement
        for epoch in range(1):  # 10 = nombre d'époques
            for i, (inputs, labels) in enumerate(train_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()


KeyboardInterrupt: 

## 6. Évaluation et analyse de résultats

In [ ]:

# Évaluation du modèle sur les données de test
model.eval()
predictions, true_labels = [], []
for inputs, labels in test_loader:
    inputs = inputs.to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    predictions.extend(predicted.tolist())
    true_labels.extend(labels.tolist())

# Afficher le rapport de classification
print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.32      0.36      0.34       103
           1       0.00      0.00      0.00         7
           2       0.81      0.29      0.43        58
           3       0.00      0.00      0.00        15
           4       0.00      0.00      0.00         2
           5       0.46      0.96      0.62       191
           6       0.00      0.00      0.00        29
           7       0.00      0.00      0.00        66
           8       0.00      0.00      0.00        60

    accuracy                           0.45       531
   macro avg       0.18      0.18      0.15       531
weighted avg       0.32      0.45      0.34       531



c:\Users\taha\Coding\ocr_test\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\taha\Coding\ocr_test\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\taha\Coding\ocr_test\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
